<a href="https://colab.research.google.com/github/gcv-tec/Examen1/blob/main/last_mile_delivery_operations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from google.colab import files 
files.upload()

In [ ]:
#Carga desde un archivo .csv sin indice y convertimos a dataframe
data = pd.read_csv('lastmile_delivery_operations_mit_lift_lab.csv')
data

In [ ]:
#Checamos la información que contiene nuestro dataframe para tener una vista general y saber como manejar de mejor
#manera los datos
data.info()


In [ ]:
#Corroboramos valores nulos
valores_nulos = data.isnull().sum()
valores_nulos

In [ ]:
#En este caso tenemos muchas variables con información completa, además hay pocas variables cuantitativas que están incompletas, por lo tanto,
# utilizaré .fillna para susituir los valores nulos del dataframe con 0, esta es la mejor opción dada la cantidad de variables y datos nulos.
#Ahora creo un filtro para separar las variables cuantitativas de las cualitativas
filtro_cuant = data.iloc[: ,[12,21,22,25,28]]
filtro_cuant

In [72]:
#Ahora limpiamos nuestras variables cuantitativas con .fillna
filtro_cuant = filtro_cuant.fillna('0')

filtro_cuant

,number_of_operators,number_of_available_trolleys,serving_customer,describe_the_accident,maximum_number_of_vehicles_in_the_traffic_jam
0,1,1,0,0,0
1,2,1,0,0,0
2,1,0,1.0,0,0
3,1,2,0,0,0
4,2,2,0,0,0
...,...,...,...,...,...
177,1,0,0,0,0
178,1,0,0,0,0
179,1,1,0,0,0
180,2,0,0,0,0


In [ ]:
#Corroboramos valores nulos de variables cuantitativas
valores_nulosn = filtro_cuant.isnull().sum()
valores_nulosn

In [43]:
#Filtro que selecciona todas las variables cualitativas
filtro_cualt = data.iloc[: , [1 ,2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 15, 17, 18, 19, 20, 23, 24, 26, 27]]


In [70]:
#Ahora limpiamos nuestras variables cualtitativas con .fillna
#filtro_cualt = filtro_cualt.fillna('0')

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 182 entries, 0 to 181
Data columns (total 29 columns):
 #   Column                                         Non-Null Count  Dtype  
---  ------                                         --------------  -----  
 0   _record_id                                     182 non-null    object 
 1   _title                                         176 non-null    object 
 2   _server_updated_at                             182 non-null    object 
 3   _updated_by                                    182 non-null    object 
 4   _geometry                                      182 non-null    object 
 5   _latitude                                      182 non-null    float64
 6   _longitude                                     182 non-null    float64
 7   arrival_of_the_freight_vehicle                 182 non-null    object 
 8   plates                                         181 non-null    object 
 9   company_if_visible                             109 non

In [ ]:
#Corroboramos valores nulos en variables cualitativas

valores_nulosl = filtro_cualt.isnull().sum()
valores_nulosl

In [ ]:
#Entonces, recapitulando, creé un filtro que me permite almacenar todas las variables cuantitativas para posteriormente purgar los datos nulos
# utilizando el comando .fillna. Ahora encontraré los outliers de mis variables cuantitativas y los reemplazaré como mejor me parezca

Outliers

In [ ]:
#Una vez que tenemos ubicadas las variables cuantitativas podemos empezar a remplazar los outliers, en este caso utilizaré el método
# de rango interquartílico por que me ofrecerá rangos de dispersión más limitados, a mi parecer esta técnica es mejor que la de desviación estándar, 
# por que en este caso quiero que los límites estén menos dispersos

In [71]:
#Verificamos la información del contenedor de las variables cuantitativas
filtro_cuant.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 182 entries, 0 to 181
Data columns (total 5 columns):
 #   Column                                         Non-Null Count  Dtype 
---  ------                                         --------------  ----- 
 0   number_of_operators                            182 non-null    int64 
 1   number_of_available_trolleys                   182 non-null    int64 
 2   serving_customer                               182 non-null    object
 3   describe_the_accident                          182 non-null    object
 4   maximum_number_of_vehicles_in_the_traffic_jam  182 non-null    object
dtypes: int64(2), object(3)
memory usage: 7.2+ KB


In [ ]:
#Corroboramos valores nulos
valores_nulosn = filtro_cuant.isnull().sum().sum()
valores_nulosn

In [ ]:
#Realizamos diagrama de caja de todas las variables cuantitativas para observar su comportamiento y comprobar que todavía hayan outliers
fig = plt.figure(figsize = (40,30))
filtro_cuant.plot(kind = 'box', vert = False)
plt.title('Valores Atípicos del DataFrame')
plt.show() #Dibujamos el diagrama

In [23]:
#Aplicamos el método de IQR para encontrar límites de valores atípicos
from numpy.lib.function_base import percentile
#Método aplicando cuartiles . Encuentro cuartiles 0.25 y 0.75
y = filtro_cuant

percentile25 = y.quantile(0.25) #Q1
percentile75 = y.quantile(0.75) #Q3
iqr = percentile75 - percentile25

Limite_Superior_iqr = percentile75 + 1.5*iqr
Limite_Inferior_iqr = percentile25 - 1.5*iqr
print('Limite Superior permitido', Limite_Superior_iqr)
print('Limite inferior permitido', Limite_Inferior_iqr)

Limite Superior permitido number_of_operators             3.5
number_of_available_trolleys    2.5
dtype: float64
Limite inferior permitido number_of_operators            -0.5
number_of_available_trolleys   -1.5
dtype: float64


In [ ]:
#Encontramos Outliers
outliers_iqr = filtro_cuant[(y<Limite_Inferior_iqr)|(y>Limite_Superior_iqr)]
outliers_iqr

In [73]:
#Encontramos Insider
data_clean_iqr = filtro_cuant[(y<=Limite_Superior_iqr)&(y>=Limite_Inferior_iqr)]
data_clean_iqr

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: Automatic reindexing on DataFrame vs Series comparisons is deprecated and will raise ValueError in a future version.  Do `left, right = left.align(right, axis=1, copy=False)` before e.g. `left == right`
  


,number_of_operators,number_of_available_trolleys,serving_customer,describe_the_accident,maximum_number_of_vehicles_in_the_traffic_jam
0,1.0,1.0,NaN,NaN,NaN
1,2.0,1.0,NaN,NaN,NaN
2,1.0,0.0,NaN,NaN,NaN
3,1.0,2.0,NaN,NaN,NaN
4,2.0,2.0,NaN,NaN,NaN
...,...,...,...,...,...
177,1.0,0.0,NaN,NaN,NaN
178,1.0,0.0,NaN,NaN,NaN
179,1.0,1.0,NaN,NaN,NaN
180,2.0,0.0,NaN,NaN,NaN


In [74]:
#Corroboramos que ya no haya valores nulos
valores_nulos = data_clean_iqr.isnull().sum().sum()
valores_nulos

558

In [75]:
data_clean_iqr  = data_clean_iqr.fillna('0')

In [76]:
#Reemplazamos valores atípicos (nulos) del dataframe con 'mean'
#Realizamos una copia del DataFrame
data_clean = data_clean_iqr.copy()
data_clean = data_clean.fillna(round(data_clean_iqr.mean(),1))
data_clean  

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  after removing the cwd from sys.path.


,number_of_operators,number_of_available_trolleys,serving_customer,describe_the_accident,maximum_number_of_vehicles_in_the_traffic_jam
0,1.0,1.0,0,0,0
1,2.0,1.0,0,0,0
2,1.0,0.0,0,0,0
3,1.0,2.0,0,0,0
4,2.0,2.0,0,0,0
...,...,...,...,...,...
177,1.0,0.0,0,0,0
178,1.0,0.0,0,0,0
179,1.0,1.0,0,0,0
180,2.0,0.0,0,0,0


In [77]:
#Concatenamos nuestros datos 

Datos_limpios = pd.concat([data_clean, filtro_cualt], axis=1)
Datos_limpios

,number_of_operators,number_of_available_trolleys,serving_customer,describe_the_accident,maximum_number_of_vehicles_in_the_traffic_jam,_title,_server_updated_at,_updated_by,_geometry,_latitude,...,type_of_vehicle,picture_of_the_parked_freight_vehicle,where_was_the_vehicle_parked,while_parked_was_the_engine_running,used_traffic_cone,vehicles_unloading_door,garage_blocking,accident,noise,traffic_congestion
0,1.0,1.0,0,0,0,73 Calle Almolonga\nPuebla de Zaragoza PUE 721...,2022-11-12 22:15:04 UTC,Salomon Abdala Perez,SRID=4326;POINT(-98.2334894708 19.0568933787),19.056893,...,Light truck or Van,https://web.fulcrumapp.com/photos/view?photos=...,Street - next to the curbside,no,no,Side,yes,no,no,no
1,2.0,1.0,0,0,0,72 Calle Almolonga\nPuebla de Zaragoza PUE 721...,2022-11-12 20:41:39 UTC,Jorge Alberto Ostorga Flores,SRID=4326;POINT(-98.233483903 19.0568494313),19.056849,...,Heavy duty truck,https://web.fulcrumapp.com/photos/view?photos=...,Street - next to the curbside,no,no,Rear,no,no,no,no
2,1.0,0.0,0,0,0,4510 Vía Volkswagen\nHeroica Puebla de Zaragoz...,2022-11-12 18:02:57 UTC,Juan Carlos Bejarano Moreno,SRID=4326;POINT(-98.2339161 19.0596775),19.059677,...,Pick-up,https://web.fulcrumapp.com/photos/view?photos=...,Street - next to the curbside,no,no,Side,no,no,no,no
3,1.0,2.0,0,0,0,4501A Avenida 2 Poniente\nHeroica Puebla de Za...,2022-11-12 17:55:44 UTC,Luis Gerardo Ramírez Lastra,SRID=4326;POINT(-98.2285674 19.0626948),19.062695,...,Light truck or Van,https://web.fulcrumapp.com/photos/view?photos=...,Street - next to the curbside,yes,no,Rear,yes,no,no,no
4,2.0,2.0,0,0,0,4717 Avenida Reforma\nHeroica Puebla de Zarago...,2022-11-12 17:55:44 UTC,Luis Gerardo Ramírez Lastra,SRID=4326;POINT(-98.231397 19.0627239),19.062724,...,Light truck or Van,https://web.fulcrumapp.com/photos/view?photos=...,Dedicated unloading area (Bay),no,no,Rear,no,no,no,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
177,1.0,0.0,0,0,0,0,2022-11-03 17:54:56 UTC,José Oropeza,SRID=4326;POINT(-98.194933077 19.043612116),19.043612,...,Light truck or Van,https://web.fulcrumapp.com/photos/view?photos=...,Street - next to the curbside,no,no,Rear,no,no,no,no
178,1.0,0.0,0,0,0,0,2022-11-03 17:54:56 UTC,José Oropeza,SRID=4326;POINT(-98.1948560508 19.0436070826),19.043607,...,Light truck or Van,https://web.fulcrumapp.com/photos/view?photos=...,Street - next to the curbside,no,no,Side,no,no,no,no
179,1.0,1.0,0,0,0,406-B Avenida 2 Oriente\nPuebla PUE 72000\nMX,2022-11-03 17:54:44 UTC,Emilio Garza,SRID=4326;POINT(-98.1945033093 19.043335537),19.043336,...,Light truck or Van,https://web.fulcrumapp.com/photos/view?photos=...,Street - next to the curbside,no,no,Rear,no,no,no,no
180,2.0,0.0,0,0,0,402E Avenida 2 Oriente\nPuebla PUE 72000\nMX,2022-11-03 17:14:09 UTC,Emilio Garza,SRID=4326;POINT(-98.1948728731 19.0435760418),19.043576,...,Pick-up,https://web.fulcrumapp.com/photos/view?photos=...,Street - next to the curbside,no,no,Rear,no,no,no,no


In [78]:
Datos_limpios.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 182 entries, 0 to 181
Data columns (total 25 columns):
 #   Column                                         Non-Null Count  Dtype  
---  ------                                         --------------  -----  
 0   number_of_operators                            182 non-null    object 
 1   number_of_available_trolleys                   182 non-null    object 
 2   serving_customer                               182 non-null    object 
 3   describe_the_accident                          182 non-null    object 
 4   maximum_number_of_vehicles_in_the_traffic_jam  182 non-null    object 
 5   _title                                         182 non-null    object 
 6   _server_updated_at                             182 non-null    object 
 7   _updated_by                                    182 non-null    object 
 8   _geometry                                      182 non-null    object 
 9   _latitude                                      182 non

In [83]:
#Converir Dataframe a csv
Datos_limpios.to_csv('Datos_limpios_last_mile.csv')

In [84]:
from google.colab import files

files.download('Datos_limpios_last_mile.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>